### Convolutional Neural Network

Convolutional Neural Network는 Convolution layer와 Pooling layer로 이루어져있다.

#### 1. 입력 정의

In [1]:
import torch
import torch.nn as nn

In [6]:
# 배치 크기 x 채널 x 높이 x 너비
inputs = torch.Tensor(1,1,28,28)
print(inputs.shape)

torch.Size([1, 1, 28, 28])



#### 2. convolution / pooling layer 선언

In [7]:
conv1 = nn.Conv2d(1, 32, kernel_size = 3, padding = 1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [8]:
conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [9]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


#### 3. 구현체를 연결하여 모델 만들기

In [10]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [11]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [12]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [13]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [14]:
# view 를 이용하여 배치 차원(첫 번째 차원)은 두고 나머지는 펼친다
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [15]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


# CNN으로 MNIST 분류하기

In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [18]:
LEARNING_RATE = 0.001
EPOCH = 15
BATCH_SIZE = 100

In [30]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(), #텐서로 변환
                         download = True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                       train=False,
                       transform = transforms.ToTensor(),
                       download = True)

In [31]:
# DataLoader를 사용하여 batch size 지정

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size = BATCH_SIZE,
                                         shuffle=True,
                                         drop_last=True)

In [33]:
# CNN 모델 설계

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2))
        
        self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2))
        
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [34]:
model = CNN().to(device)

In [35]:
# 비용함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss().to(device)  # 비용 함수에 softmax 함수 포함되어 있음
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [36]:
total_batch = len(data_loader)

In [37]:
for epoch in range(EPOCH):
    avg_cost = 0
    
    for X,Y in data_loader:    #X는 mini batch, Y는 label
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

[Epoch :    1] cost = 0.227034941
[Epoch :    2] cost = 0.0616304688
[Epoch :    3] cost = 0.045381587
[Epoch :    4] cost = 0.0358436704
[Epoch :    5] cost = 0.0295987371
[Epoch :    6] cost = 0.0246207658
[Epoch :    7] cost = 0.0208311938
[Epoch :    8] cost = 0.0179441292
[Epoch :    9] cost = 0.014555526
[Epoch :   10] cost = 0.0116540259
[Epoch :   11] cost = 0.0102349715
[Epoch :   12] cost = 0.00953598972
[Epoch :   13] cost = 0.0086209951
[Epoch :   14] cost = 0.00646122731
[Epoch :   15] cost = 0.00579035934


- 테스트를 해보자. 학습을 진행하는 것이 아니므로 torch.no_grad() 사용

In [41]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct = torch.argmax(prediction, 1) == Y_test
    accuracy = correct.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.9861000180244446
